In [ ]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
# import uuid

## Create bucket & Validation Region for S3

In [ ]:
bucket_name = 'aps360project' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-2


## Create Paths to S3 Buckets for storage of model data

In [ ]:
# Prefix for files in bucket
prefix = 'chest_x_ray_classification'

# Dataset directory
dataset = 'Xray_Dataset'

# Model output folder name
output_dir_name = 'trial_7'

# S3 Path bucket to get the data for training (Train, Test, Validation)
dataset_dir = 's3://{}/{}/{}'.format(bucket_name, prefix, dataset)

# output path for SageMaker to dump all model artifacts and graphs etc
output_dir = 's3://{}/{}/{}'.format(bucket_name, prefix, output_dir_name)

# # checkpoints for spot training
# checkpoint_suffix = str(uuid.uuid4())[:8]
# checkpoint_s3_path = 's3://{}/{}/{}/checkpoint-{}'.format(bucket_name, prefix, output_dir_name, checkpoint_suffix)

# sanity check for output path for model data
print('Dataset directory <dataset_dir>: ', dataset_dir)
print('Model Output directory <output_dir>: ', output_dir)
# print('Checkpointing Path: <checkpoint_s3_path>: {}'.format(checkpoint_s3_path))


Dataset directory <dataset_dir>:  s3://aps360project/chest_x_ray_classification/Xray_Dataset
Model Output directory <output_dir>:  s3://aps360project/chest_x_ray_classification/trial_7


## Manage Spot Training

In [ ]:
# use_spot_instances = True
# max_run=24*60*60
# max_wait = 24*60*60

In [ ]:
# initialize hyperparamters
hyperparameters = {
    'epochs': 6,
    'batch-size': 256,
    'learning-rate': 0.001 
}

# Training instance
training_instance = 'ml.g4dn.xlarge'

# Create the current role to use sagemaker 
role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.pytorch import PyTorch

# Create a Pytorch estimator to run the training script on AWS Sagemaker
estimator = PyTorch(
                entry_point='trial7xray.py',
                role=role,
                framework_version='1.8.0',
                py_version='py3',
                output_path=output_dir,
                train_instance_count=1,
                script_mode=True,
                train_instance_type=training_instance,
                hyperparameters= hyperparameters
            )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
estimator.fit({'training': dataset_dir})

S3UploadFailedError: Failed to upload /tmp/tmp9dd3tnkf/source.tar.gz to aps360project/pytorch-training-2021-04-02-20-00-15-403/source/sourcedir.tar.gz: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied